In [ ]:
# %pip install folium

In [ ]:
import folium
from folium import plugins
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopy.distance

%matplotlib inline

In [ ]:
# change to root directory of respository
%cd C:\Users\saara\Documents\_Grad_University\Y1S1\CMPUT692\term_project\flexible-cells-algorithm

In [ ]:
def pred_centers(txt):  # converts the output of true centers into manipulatable lists
    min_max_lst = []
    txt = txt.split("\n")
    for line in txt:
        if line != "" and line[0]=="C":
            lst = line.split("),")[0][10:].split(", ")
            to_append = [float(lst[0]), float(lst[1]), float(lst[3]), float(lst[4])]
            min_max_lst.append(to_append)
    return min_max_lst

def tru_centers(txt):  # converts the output of predicted centers into manipulatable lists
    coords_list = []
    txt = txt.split("\n")
    for line in txt:
        if line != "":
            lst = line.split("(")[1].replace(")","").split(", ")
            to_append = [float(lst[0]), float(lst[1])]
            coords_list.append(to_append)
    return coords_list

In [ ]:
event_coords_filepath = "data/synthetic_coords.txt"
map_name = "visualizations/synthetic_interactive_map.html"

In [ ]:
# replace with output of our center finding algorithm
pred_cntrs = '''
Center: ((-108.2875, -105.8875, 2.4, -81.7625, -84.1625, 2.4, 4), 800)
(C,alpha): (800, 4.999996371653228)
Center: ((-80.3625, -77.9625, 2.4, 14.275, 11.875, 2.4, 2), 800)
(C,alpha): (800, 6.550317628686485e-06)
Center: ((73.725, 76.125, 2.4, -81.75, -84.15, 2.4, 1), 800)
(C,alpha): (800, 4.999996371653228)
Center: ((153.675, 156.075, 2.4, 46.2125, 43.8125, 2.4, 3), 801)
(C,alpha): (801, 4.999996371653228)
'''

In [ ]:
# replace with output of Backstrom's center finding algorithm
backstrom_cntr = """
Center: ((154.1, 154.2, 0.1, 45.5, 45.6, 0.1), 6)
(C,alpha): (6, 4.999996371653228)
"""

In [ ]:
# replace with output of our data generation step
tru_cntrs = '''
Center 1: (-83.0, 75.0)
Center 2: (45.0, 155.0)
Center 3: (-83.0, -107.0)
Center 4: (13.0, -79.0)
'''

In [ ]:
# creates lat and long of backstrom center
back_out = pred_centers(backstrom_cntr)[0]
back_coords = ((back_out[2]+back_out[3])/2,(back_out[0]+back_out[1])/2)

In [ ]:
back_coords

In [ ]:
# contains coordinates of observations
coords_df = pd.read_csv(event_coords_filepath, header=None)  # ground truth observations

In [ ]:
coords_df.head()

In [ ]:
# contains coordinates of true centers
tru_df = pd.DataFrame(tru_centers(tru_cntrs), columns=["latitude_true", "longitude_true"])

In [ ]:
tru_df.head()

In [ ]:
# creates dataframe containing predicted and true center information
centers_df = pd.DataFrame(pred_centers(pred_cntrs), columns=["min_x", "max_x", "min_y", "max_y"])
centers_df["longitude_guess"] = (centers_df.min_x + centers_df.max_x)/2
centers_df["latitude_guess"] = (centers_df.min_y + centers_df.max_y)/2
centers_df["longitude_true"] = tru_df.longitude_true
centers_df["latitude_true"] = tru_df.latitude_true

In [ ]:
centers_df.head()

In [ ]:
# create map
m = folium.Map([39, -98], zoom_start=4)

# # plots all events as small dots
# for index, row in coords_df.iterrows():
#     folium.CircleMarker([row[0], row[1]],
#                         radius=1,
# #                         popup=row['name'],
#                         fill_color="#3db7e4", # divvy color
#                        ).add_to(m)

# plots our multiple center guesses
for index, row in centers_df.iterrows():
    folium.CircleMarker([row["latitude_guess"], row["longitude_guess"]],
                        radius=3,
                        fill=True, # Set fill to True
                        popup=("Our Guess"),
                        color = '#2e43a6').add_to(m)
    
# plot Backstrom's single center guess   
folium.CircleMarker(back_coords,
                    radius=3,
                    fill=True, # Set fill to True
                    popup=("Backstrom's Guess"),
                    color = '#FF0000').add_to(m)

# plot ground truth centers
for index, row in centers_df.iterrows():
    folium.CircleMarker([row["latitude_true"], row["longitude_true"]],
                        radius=3,
                        fill=True, # Set fill to True
                        popup=("Ground Truth"),
                        color = '#9d2fad').add_to(m)
    
# plot heatmap
m.add_child(plugins.HeatMap(coords_df, radius=15))

m

In [ ]:
# saves plot as an interactive map
m.save(map_name)

In [ ]:
# distance from backstrom to all centers
lst = []
for i in centers_df.index:
    lst.append(round(geopy.distance.distance((centers_df.latitude_true[i], centers_df.longitude_true[i]), back_coords).miles,2))
lst    

In [ ]:
#matrix of distances from our centers to true centers
mat = [] # each row is a guess, each col is the truth
for i in centers_df.index: 
    apnd = []
    for j in centers_df.index:
        coords_pred = (centers_df.latitude_guess[i], centers_df.longitude_guess[i])
        coords_true = (centers_df.latitude_true[j], centers_df.longitude_true[j])
        apnd.append(round(geopy.distance.distance(coords_pred, coords_true).miles,2))
    mat.append(apnd)
mat